In [117]:
import numpy as np
import scipy
from scipy.stats import randint
np.random.seed(1)


In [118]:
NUM_PLAYERS = 4
NUM_DEEDS = 24
NUM_DEED_TYPES = NUM_DEEDS // 3
NUM_ANTI_TRUST = 3
INFINITE_VALUE = 1000000
ANTI_TRUST_FINE = 20
MIN_BID = 5
RESORT_COST = 20
HOTEL_COST = 10


In [119]:
class Player(object):
    def __init__(self, name):
        self.cash = 150
        self.deeds = set()
        self.hotels = set()
        self.resorts = set()
        self.name = name
        self.game_over = False
        
    def countType(self, deed_type):
        count = 0
        for deed in self.deeds:
            if deedType(deed) == deed_type:
                count += 1
        return count
    
    def gameOver(self):
        self.deeds = set()
        self.hotels = set()
        self.resorts = set()
        self.name = "XX" + self.name + "XX"
        self.game_over = True

def deedType(deed):
    if deed >= NUM_DEEDS:
        raise Exception("Invalid card")
    return deed // 3

def rentCost(deed,owner):
    deed_type = deedType(deed)
    if deed_type in owner.resorts:
        return 25
    if deed_type in owner.hotels:
        return 10
    owned = owner.countType(deed_type)
    if owned == 0:
        return 0
    elif owned == 1:
        return 1
    elif owned == 2:
        return 2
    elif owned == 3:
        return 5
    else:
        print(self.deeds)
        raise Exception("Oops " + str(owned))
    
def incPlayer(i):
    return (i+1)%NUM_PLAYERS

In [120]:
# All of the AI can be derived from one question: What is a deed worth?

def computeDeedValue(deed,buyer_index,players):
    player = players[buyer_index]
    already_owns = False
    if deed in player.deeds:
        deed_type = deedType(deed)
        if deed_type in player.hotels or deed_type in player.resorts:
            return INFINITE_VALUE # Can't get rid of these, so assume high value
        already_owns = True
        player.deeds.remove(deed)

    valueWithoutDeed = rentCost(deed,player)*8
    player.deeds.add(deed)
    valueWithDeed = rentCost(deed,player)*8
    if player.countType(deedType(deed)) == 3:
        valueWithDeed = 7*8 # hack to account for the fact that now the player has the opportunity to buy a hotel.
    if not already_owns:
        player.deeds.remove(deed)
    return valueWithDeed - valueWithoutDeed
    

In [121]:
players = []
for x in range(NUM_PLAYERS):
    players.append(Player("Player_" + str(x + 1)))

visit_cards = np.arange(NUM_DEEDS + NUM_ANTI_TRUST)
on_card = 0
np.random.shuffle(visit_cards)

def auction(deed, seller, first_to_bid):
    top_bid = None
    second_highest_bid = None
    player_bid_map = {}
    for x in range(NUM_PLAYERS):
        if x != seller:
            player_bid_map[x] = 0
    player_to_bid = first_to_bid
    while len(player_bid_map)>1:
        if player_to_bid not in player_bid_map:
            player_to_bid = incPlayer(player_to_bid)
            continue
            
        # TODO: Consider the value to the other players in the game
        value_to_player = min(players[player_to_bid].cash, computeDeedValue(deed, player_to_bid, players))
        if value_to_player < MIN_BID:
            # Fold
            del player_bid_map[player_to_bid]
        elif top_bid is not None and value_to_player <= top_bid:
            # Fold
            del player_bid_map[player_to_bid]
        else:
            # Bid
            second_highest_bid = top_bid
            top_bid = value_to_player
            player_bid_map[player_to_bid] = value_to_player
        player_to_bid = incPlayer(player_to_bid)
        
    if len(player_bid_map)==0:
        return # No one bought
    else:
        winning_player_index = list(player_bid_map)[0]
        winning_player_bid = player_bid_map[winning_player_index]
        if winning_player_bid == 0:
            return # No one bought
        players[winning_player_index].cash -= winning_player_bid
        players[winning_player_index].deeds.add(deed)
        if seller:
            players[seller].cash += winning_player_bid
        print(players[winning_player_index].name,'bid',winning_player_bid,'for',deed)

for turn in range(100):
    for player_index in range(len(players)):
        player = players[player_index]

        if player.cash < 0:
            continue

        # Shuffle the visit deck
        if on_card >= visit_cards.shape[0] // 2:
            on_card = 0
            np.random.shuffle(visit_cards)
            # When we shuffle, give each player some money
            print('Shuffle, everyone gets money!')
            for p in players:
                p.cash += 1
        # Draw a card
        card = visit_cards[on_card]
        on_card += 1
        if card >= NUM_DEEDS:
            # Anti-trust card.  Loop through player's deeds and try to sell one
            worst_value = -1
            worst_deed = None
            for deed in player.deeds:
                value = computeDeedValue(deed,player_index,players)
                if value == INFINITE_VALUE:
                    continue
                if worst_value == -1 or worst_value > value:
                    worst_value = value
                    worst_deed = deed
            if worst_deed is None:
                # Can't sell anything, pay a fine
                player.cash -= ANTI_TRUST_FINE
                print(player.name,'charged anti-trust fine of',ANTI_TRUST_FINE)
            else:
                # Sell the least valuable card (TODO: Consider the value to other players)
                player.deeds.remove(worst_deed)
                print(player.name,'forced to auction',worst_deed)
                auction(worst_deed,player_index,incPlayer(player_index))
        else:
            owned = False
            for p in players:
                if card in p.deeds:
                    owned = True
                    if p == player:
                        continue # Landed on your own card
                    # Landed on someone else's card, need to pay a fee
                    rent_cost = rentCost(card,p)
                    player.cash -= rent_cost
                    p.cash += rent_cost
                    print(player.name,'charged',rent_cost,'rent on',deed,'by',p.name)
            if not owned:
                auction(card,None,player_index)

        if player.cash < 0 and player.game_over == False:
            # Player is knocked out of the game
            print(player.name,'is knocked out of the game')
            player.gameOver()
            
        # Check if game is over
        players_alive = 0
        for p in players:
            if p.cash >= 0:
                players_alive += 1
        if players_alive < 2:
            break

        if not player.game_over:
            # Opportunity to buy hotel/resort
            for deed_type in range(NUM_DEED_TYPES):
                if deed_type in player.resorts:
                    continue # Nothing more to buy
                elif deed_type in player.hotels:
                    # Opportunity to upgrade to resort
                    if player.cash > RESORT_COST:
                        print(player.name,'bought a resort on',deed_type)
                        player.cash -= RESORT_COST
                        player.hotels.remove(deed_type)
                        player.resorts.add(deed_type)
                elif player.countType(deed_type)==3:
                    # Opportunity to buy hotel
                    if player.cash > HOTEL_COST:
                        print(player.name,'bought a hotel on',deed_type)
                        player.cash -= HOTEL_COST
                        player.hotels.add(deed_type)
        
    print("TURN",turn)
    for player in players:
        print(player.name,player.cash,player.deeds,player.hotels,player.resorts)
    # Check if game is over
    players_alive = 0
    for p in players:
        if p.cash >= 0:
            players_alive += 1
    if players_alive < 2:
        print('GAME OVER')
        break



Player_1 bid 8 for 17
Player_2 bid 8 for 14
Player_3 bid 8 for 18
Player_4 bid 8 for 3
TURN 0
Player_1 142 {17} set() set()
Player_2 142 {14} set() set()
Player_3 142 {18} set() set()
Player_4 142 {3} set() set()
Player_1 bid 8 for 23
Player_2 bid 8 for 10
Player_3 bid 8 for 20
Player_4 bid 8 for 21
TURN 1
Player_1 134 {17, 23} set() set()
Player_2 134 {10, 14} set() set()
Player_3 134 {18, 20} set() set()
Player_4 134 {3, 21} set() set()
Player_1 bid 8 for 4
Player_2 bid 8 for 2
Player_3 bid 8 for 22
Player_4 bid 8 for 6
TURN 2
Player_1 126 {17, 4, 23} set() set()
Player_2 126 {2, 10, 14} set() set()
Player_3 126 {18, 20, 22} set() set()
Player_4 126 {3, 21, 6} set() set()
Player_3 bid 40 for 19
Shuffle, everyone gets money!
Player_2 bid 8 for 5
Player_3 bid 8 for 12
Player_3 bought a hotel on 6
Player_4 bid 8 for 1
TURN 3
Player_1 127 {17, 4, 23} set() set()
Player_2 119 {2, 5, 10, 14} set() set()
Player_3 69 {18, 19, 20, 22, 12} {6} set()
Player_4 119 {1, 3, 21, 6} set() set()
Playe